In [1]:
!pip install numpy
!pip install pandas
!pip install tensorflow
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import precision_score, recall_score

**Step 1:** Upload the preprocessed data that contains only form names and fieldlabels

In [3]:

#df = pd.read_csv('c:/Users/migle/Desktop/BPR/IPWFormAi/data/all_data.csv', on_bad_lines='skip')
df = pd.read_csv('all_datav3.csv', delimiter=';', on_bad_lines='skip')
#handle blocktype(removeit).
indices_to_drop = df[df['elementtype'] == 'block'].index
df = df.drop(indices_to_drop)

#put specialtype in place.
for index, row in df.iterrows():
    if row['elementtype'] == 'special' and pd.isnull(row['fieldtype']):  # Check if 'elementtype' is empty
        df.at[index, 'fieldtype'] = row['specialtype']  # Assign 'specialtype' value to 'elementtype'

df = df.drop(columns=['Kunde','fieldtype','customform','specialtype','elementtype', 'customname','fieldobjectid','fieldparentid','fieldparenttype','blocktype','fieldrelation','language'])
display(df)

,name,fieldlabel
0,Kundereklamation,Vælg kunde
2,Kundereklamation,Gadenavn
3,Kundereklamation,Postnr
4,Kundereklamation,By
5,Kundereklamation,Telefonnummer
...,...,...
9446,SI - Handleplan,Evt. anden strategisk initiativ
9447,SI - Aktiviteter,Fremdrift
9448,SI - Aktiviteter,ID
9449,SI - Handleplan,ID


**Step 2:** Additional step in data preprocessing is concatinatting all fieldlabels to a form name so that they stand all together per form name

In [4]:

# Step 1: Group by 'Form name' and aggregate the field labels into a list
grouped = df.groupby('name')['fieldlabel'].apply(list).reset_index()

# Step 2: Expand the lists into separate columns (using pd.DataFrame to handle different numbers of labels)
expanded = pd.DataFrame(grouped['fieldlabel'].tolist())

# Step 3: Combine the 'Form name' column with the expanded field labels
result = pd.concat([grouped['name'], expanded], axis=1)

# Step 4: Rename the columns appropriately
result.columns = ['name'] + [f'fieldlabel {i+1}' for i in range(result.shape[1] - 1)]

display(result)

,name,fieldlabel 1,fieldlabel 2,fieldlabel 3,fieldlabel 4,fieldlabel 5,fieldlabel 6,fieldlabel 7,fieldlabel 8,fieldlabel 9,...,fieldlabel 238,fieldlabel 239,fieldlabel 240,fieldlabel 241,fieldlabel 242,fieldlabel 243,fieldlabel 244,fieldlabel 245,fieldlabel 246,fieldlabel 247
0,*_270224_OBT - Alarmering,Emne,Link,Læst og forstået,Ikke forstået,Instruktør,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,*_270224_OBT - Andet,Emne,Link,Ikke forstået,Læst og forstået,Instruktør,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,*_270224_OBT - Beredskab,Link,Emne,Instruktør,Ikke forstået,Læst og forstået,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,*_270224_OBT - Brandbekæmpelse,Link,Emne,Instruktør,Ikke forstået,Læst og forstået,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,*_270224_OBT - Førstehjælp og udstyr,Link,Emne,Instruktør,Ikke forstået,Læst og forstået,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,amnj-afvigelsesblanket,Afvigelses rapport nr.,Løbe nr.,Vare nr.,Varenavn,Batch nr.,Udstyrstype,ID-nr.,Dok. nr./udg. nr.,Dok. navn,...,None,None,None,None,None,None,None,None,None,None
734,amnj-fejltype,Tekst,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
735,amnj-kundereklamation,Kunde navn,Gadenavn,Post nr.,By,Tlf. nr.,E-mail,Reklamations nr.,Kontaktperson,Vare nr.,...,None,None,None,None,None,None,None,None,None,None
736,amnj-leverandørreklamation,Reklamations nr.,Leverandørnavn,Gadenavn,Post nr.,By,Tlf. nr.,E-mail,Kontaktperson,Vare nr.,...,None,None,None,None,None,None,None,None,None,None


In [5]:
y = pd.get_dummies(df['fieldlabel'])

y = df.pivot_table(index='name', columns='fieldlabel', aggfunc=lambda x: 1, fill_value=0)

# Resetting index to get 'Form name' as a column again
y = y.reset_index()

# Step 3: Define X (Form names)
X = y['name']  # Form names as input

# Step 4: Remove 'Form name' from y after defining X
y = y.drop(columns=['name'])  # Only the field labels remain in y



In [6]:
#make sure that we are working with more than 1 of unique classes in fieldlabels so that we can use 'stratify' when splitting data

unique_classes_per_column = y.nunique()
columns_to_keep = unique_classes_per_column[unique_classes_per_column > 1].index
y_filtered = y[columns_to_keep]


In [7]:
print(X.shape)
print(y_filtered.shape)

(738,)
(738, 4106)


In [8]:
# Ensure both X and y_filtered have the same index
X = X.reset_index(drop=True)
y_filtered = y_filtered.reset_index(drop=True)

In [9]:
dfVal = pd.read_csv('all_data-validationCSV.csv', delimiter=';', on_bad_lines='skip')
indices_to_drop = dfVal[dfVal['elementtype'] == 'block'].index
dfVal = dfVal.drop(indices_to_drop)

#put specialtype in place.
for index, row in dfVal.iterrows():
    if row['elementtype'] == 'special' and pd.isnull(row['fieldtype']):  # Check if 'elementtype' is empty
        df.at[index, 'fieldtype'] = row['specialtype']  # Assign 'specialtype' value to 'elementtype'

dfVal = dfVal.drop(columns=['Kunde','fieldtype','customform','specialtype','elementtype', 'customname','fieldobjectid','fieldparentid','fieldparenttype','blocktype','fieldrelation','language'])
display(dfVal)
# Step 1: Group by 'Form name' and aggregate the field labels into a list
grouped = dfVal.groupby('name')['fieldlabel'].apply(list).reset_index()

# Step 2: Expand the lists into separate columns (using pd.DataFrame to handle different numbers of labels)
expanded = pd.DataFrame(grouped['fieldlabel'].tolist())

# Combine grouped and expanded
dataVal = pd.concat([grouped['name'], expanded], axis=1)

# Rename columns appropriately
dataVal.columns = ['name'] + [f'fieldlabel {i+1}' for i in range(dataVal.shape[1] - 1)]

# Display the result
display(dataVal)

,name,fieldlabel
0,Leverancer,Er lÃ¸sningen Hostet?
1,Forecast,Dato
2,Forecast,Tilbudssum: Tilbud >= 50% Deadline <90 dage
3,Forecast,Sum ordrebeholdning (Tilgode)
4,Forecast,Sum ordrebeholdning (Est.tilgode)
...,...,...
880,Intern IT Support,"Registrerer du tid, kan du bruge flg. opg.nr: ..."
881,Azure - UdlÃ¸b af client secret,Ansvarlig for opsÃ¦tning
883,Leverancer,Installationen oprettes pÃ¥ domÃ¦net xxxx.ipw.dk
884,Leverancer,Installationen oprettes pÃ¥ domÃ¦net <b>xxxx.i...


,name,fieldlabel 1,fieldlabel 2,fieldlabel 3,fieldlabel 4,fieldlabel 5,fieldlabel 6,fieldlabel 7,fieldlabel 8,fieldlabel 9,...,fieldlabel 24,fieldlabel 25,fieldlabel 26,fieldlabel 27,fieldlabel 28,fieldlabel 29,fieldlabel 30,fieldlabel 31,fieldlabel 32,fieldlabel 33
0,2024-01-08 - JS Test,Test,Billede,Tjek,F_COMBO_MUL felt,F_REL_MUL felt,Indlejret,URL,Signatur,bRUGERE,...,None,None,None,None,None,None,None,None,None,None
1,APV 2021 Arbejde hjemmefra,Hvor tit arbejder du hjemmefra nu om ugen (i g...,Hvor meget forventer du at arbejde hjemmefra e...,FÃ¸lgende spÃ¸rgsmÃ¥l handler om din arbejdspl...,Hvor stor er afstanden fra dine Ã¸jne til skÃ¦...,Har du frit benplads under dit arbejdsbord?,Benplads,Hvor meget afstand er der bag forkanten af dit...,Hvor meget benplads har du i bredden? Dvs hvor...,Kan dit bord indstilles i hÃ¸jde?,...,Holdning,Se ogsÃ¥ billederne under spÃ¸rgeskemaet og br...,Kommentar,Du skal tage udgangspunkt i din almindelige ar...,HvornÃ¥r oplever du en dÃ¥rlig belysning?,Oplever du at du har den rigtige arbejdsstilling?,Oprettet af,Ansvarlig,Vurdering AMG,Kommentar
2,APV tjekliste kontor,Hvor meget bevÃ¦ger du dig i lÃ¸bet af dagen?,Har du ofte for mange opgaver eller for travlt...,Mangler du hjÃ¦lp / stÃ¸tte fra din ledelse og...,"Oplever man mobning, seksuel chikane, diskrimi...",Er der for varmt eller koldt i arbejdsrummene ...,"Er luften i lokalerne dÃ¥rlig, f.e. pÃ¥ grund ...",Evt. uddybbelse,Er stÃ¸jen i mere end 2 timer om arbejdsdagen ...,"Er der risiko for, at I kan falde eller snuble...",...,None,None,None,None,None,None,None,None,None,None
3,Audit,Findings (brug faneblad),Beskrivelse,Nr.,Skabelon,Ã…rshjulspunkter,Filer,Stadie,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Audit skabelon,Type,Emne/omrÃ¥de,Beskrivelse,Kommentar,Ã…rshjulskontrolpunkter,Hyppighed,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,Vareoversigt,Vare,Tilbudstekst,Tilbudsskabelon DK,Aktiveres via konfiguration,Vejledning,Har sÃ¦rskilt tilbudsskabelon,Website DK,Website EN,Supportsite,...,None,None,None,None,None,None,None,None,None,None
126,Versioner,Versionnr,Versionskommentar til bruger (dansk),Versionskommentar til bruger (engelsk),Erstatter version,PHP Extension,Godkendt at QA,Forventet udgivelsesdato,Jira release klar,Demo for konsulenter afholdt,...,None,None,None,None,None,None,None,None,None,None
127,Vidensdeling,Emne,Uddybende beskrivelse,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
128,decimaltester,Input 1,Input 2,Output,Audit inspection score,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


**Step 3:** Splitting our data into train and test.

In our case, stratify does not work. Could be due to imbalanced labels and classes do not have enough samples. By not being able to use stratify when splitting data it can provide a challenge to ensure that train and test data maintain the proportion as in the original dataset. 

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y_filtered, test_size=0.2, random_state=42)

Using a random forest classifier to be able to capture more complex relationships in the data. Using Bag of Words because our input data is text (form names)

In [11]:
#Define the multioutput classifier using BoW 
from sklearn.ensemble import RandomForestClassifier

model = make_pipeline(CountVectorizer(), MultiOutputClassifier(RandomForestClassifier()))


When trying to fit the data into the model, it complains about the number of classes. Therefore, we are performing additional steps to ensure the correct number of unique classes across the dataset. 

In [12]:
# Checking for columns with only one unique value
single_class_labels = y_train.columns[y_train.nunique() == 1]
print(f"Columns with only one class in y_train: {single_class_labels}")


Columns with only one class in y_train: Index(['-',
       '0.1 Er det uklart, hvem der står for oplæring og instruktion af nye medarbejdere og hvad oplæringen og instruktionen omfatter?',
       '0.2 Er det uklart for den nyansatte, hvilke opgaver han/hun kan og må udføre, så der ikke opstår risiko for hans/hendes sikkerhed og sundhed?',
       '0.3 Er der nyansatte, der ikke har fået vejledning og instruktion i, hvordan de kan udføre arbejdet forsvarligt?',
       '0.4 Mangler der tilsyn med, om arbejdet bliver udført forsvarligt?',
       '0.5 Er der medarbejdere, der ikke har fået vejledning og instruktion i, hvordan de kan udføre arbejdet forsvarligt, hvis der sker ændringer i arbejdsopgaver, udstyr, teknologi eller stoffer og materialer?',
       '0.6 Er det uklart for medarbejderne, hvordan sikkerhed prioriteres i forhold til fx hensyn til leveringstid, effektivitet, service og kvalitet?',
       '0.7 Er det uklart for medarbejderne, hvad de skal gøre, hvis der opstår uforudsete

In [13]:
# Remove columns with only one class
y_train_filtered = y_train.drop(columns=single_class_labels)
y_test_filtered = y_test.drop(columns=single_class_labels)


**Step 4:** Fitting the data to the model and checking accuracy

In [14]:
model.fit(X_train, y_train_filtered)
#y_pred = model.predict(X_test)
#y_pred_train = model.predict(X_train)

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('multioutputclassifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

In [15]:
def predict_fieldlabels(model, form_name, y_columns):
    # Predict probabilities (since MultiOutputClassifier returns binary outputs)
    prediction = model.predict([form_name])
    print(prediction)
    
    # Convert predictions to fieldlabel names
    predicted_fieldlabels = [label for idx, label in enumerate(y_columns) if prediction[0][idx] == 1]
    print(predicted_fieldlabels)
    
    return predicted_fieldlabels

# Example usage
form_name_example = "ar"
predicted_labels = predict_fieldlabels(model, form_name_example, y_train_filtered.columns)
print(f"Predicted fieldlabels for '{form_name_example}': {predicted_labels}")

[[0 0 0 ... 0 0 0]]
[]
Predicted fieldlabels for 'ar': []


In [27]:
maxPoints = 0;
overalPoints = 0;
negativePoints = 0;
fieldlabels = [];

# Grab form names of Validation Set
#print(dataVal)
for row in dataVal.index:
     # Print the entire row
    #print(f"Row {row}:")
    #print(dataVal.loc[row, "name"])  # Use .loc to access the row by its index
    form_name = dataVal.loc[row, "name"]
    predicted_labels = predict_fieldlabels(model, form_name, y_train_filtered.columns)

    #print(dataVal.loc[row, "fieldlabel"])
    #print("-----------")
    #print(predicted_labels)
    #print(dataVal)

    for i in range(1, 33):
        fieldLabelName = f"fieldlabel {i}";
        fieldlabels.append(dataVal.loc[row, fieldLabelName])

    for predLabel in predicted_labels:
        maxPoints += 1
        #print(predLabel.split(","))
        #print(label)
        #print(predLabel)
        if(predLabel in fieldlabels):
            overalPoints += 1;
            print("matches")
        else:
             negativePoints += 1;    
    print(f"Predicted Captures for '{form_name}': {predicted_labels}")


print(maxPoints)
print(overalPoints)
print(negativePoints)
percentage = overalPoints/maxPoints;
precision = overalPoints / (overalPoints + negativePoints)
precision = precision * 100;
percentage = percentage * 100;
print(f"Precision:'{precision}'")
print(f"percentage:'{percentage}'")

[[0 0 0 ... 0 0 0]]
[]
Predicted Captures for '2024-01-08 - JS Test': []
[[0 0 0 ... 0 0 0]]
[]
Predicted Captures for 'APV 2021 Arbejde hjemmefra': []
[[0 0 0 ... 0 0 0]]
[]
Predicted Captures for 'APV tjekliste kontor': []
[[0 0 0 ... 0 1 0]]
['Accept af findings', 'Actual improvement?', 'Afdeling', 'Afdeling der skal auditeres', 'Afdelingsansvarlig', 'Afvigelse i henhold til?', 'Afvigelser', 'Ansvarlig', 'Ansvarlig leder', 'Antal afvigelser', 'Antal forbedringsforslag', 'Assisterende auditor', 'Assisting auditor', 'Attach files', 'Audit Område', 'Audit area', 'Audit result', 'Auditdato', 'Auditor', 'Auditrapport er vedhæftet?', 'Auditresultat - resumé', 'Bemærkninger', 'Blank linje', 'Comments', 'Created', 'Created by', 'Date of audit', 'Dato', 'Deltagere', 'Deltagere (Ledere og medarbejdere)', 'Ekstern auditør', 'Emne/område', 'Er der afvigelser', 'Er der forbedringsforslag?', 'Evt. præcisering af afdeling', 'Evt. relateret afvigelser', 'External auditor', 'Follow-up performed', 'F

**Training set performance**
- Precision --:-- indicates that model identifies nearly all true positives

**Test set performance**
- Precision 12.17
- Difference between train and test set metrics suggests overfitting, where model memorized the training data but fails to adjust to the new examples.

**Step 5:** Finetuning the hyperparameters

Using a parameter to automatically adjust the weights which should improve the overall recall and precision for imbalanced classes.

In [28]:
model = make_pipeline(
    CountVectorizer(),
    MultiOutputClassifier(
        RandomForestClassifier(n_estimators=100, max_depth=10, max_features='sqrt', class_weight='balanced', random_state=42)
    )
)

In [29]:
# Fit the model on the training data
model.fit(X_train, y_train_filtered)

# Make predictions
y_pred = model.predict(X_test)
y_pred_train = model.predict(X_train)

In [30]:
maxPoints = 0;
overalPoints = 0;
negativePoints = 0;
fieldlabels = [];

# Grab form names of Validation Set
#print(dataVal)
for row in dataVal.index:
     # Print the entire row
    #print(f"Row {row}:")
    #print(dataVal.loc[row, "name"])  # Use .loc to access the row by its index
    form_name = dataVal.loc[row, "name"]
    predicted_labels = predict_fieldlabels(model, form_name, y_train_filtered.columns)

    #print(dataVal.loc[row, "fieldlabel"])
    #print("-----------")
    #print(predicted_labels)
    #print(dataVal)

    for i in range(1, 33):
        fieldLabelName = f"fieldlabel {i}";
        fieldlabels.append(dataVal.loc[row, fieldLabelName])

    for predLabel in predicted_labels:
        #print(predLabel.split(","))
        #print(label)
        #print(predLabel)
        if(predLabel in fieldlabels):
            overalPoints += 1;
            print("matches")
        else:
             negativePoints += 1;    
    print(f"Predicted Captures for '{form_name}': {predicted_labels}")


print(maxPoints)
print(overalPoints)
print(negativePoints)
percentage = overalPoints/maxPoints;
precision = overalPoints / (overalPoints + negativePoints)
precision = precision * 100;
percentage = percentage * 100;
print(f"Precision:'{precision}'")
print(f"percentage:'{percentage}'")

[[0 0 0 ... 0 0 0]]
[]
Predicted Captures for '2024-01-08 - JS Test': []
[[0 0 0 ... 0 0 0]]
[]
Predicted Captures for 'APV 2021 Arbejde hjemmefra': []
[[0 0 0 ... 0 0 0]]
[]
Predicted Captures for 'APV tjekliste kontor': []
[[0 0 0 ... 0 0 0]]
['Assisterende auditor', 'Audit Område', 'Auditdato', 'Deltagere', 'Ledende auditor', 'Nummer', 'Opfølgning foretaget', 'Reel forbedring?', 'Send info til brugere', 'Send info til brugergrupper', 'Vedhæft filer']
Predicted Captures for 'Audit': ['Assisterende auditor', 'Audit Område', 'Auditdato', 'Deltagere', 'Ledende auditor', 'Nummer', 'Opfølgning foretaget', 'Reel forbedring?', 'Send info til brugere', 'Send info til brugergrupper', 'Vedhæft filer']
[[0 0 0 ... 0 0 0]]
['Assisterende auditor', 'Audit Område', 'Auditdato', 'Deltagere', 'Ledende auditor', 'Nummer', 'Opfølgning foretaget', 'Reel forbedring?', 'Send info til brugere', 'Send info til brugergrupper', 'Vedhæft filer']
Predicted Captures for 'Audit skabelon': ['Assisterende auditor'

**Test set results**
Precision of test set: 28.83

**Train set results**
Precision of train set: 0.64

 **Summary from finetuning**

- The finetuning improved the performance regarding precision and recall of the training data
- The model has improved quite a bit in handling imblanaces, but its still not perfect
- Precision 0.64 on train data tells us that model is very good at avoiding false positives
- Recall 0.6 tells us that model correctly identified 60% of the actual positive field labels (40% of false negatives)

### **Summary of Experiment 1:**

- Used Bag of Words in Multi-output classifier combined with Random Forest Classifier to handle complex relationships in the dataset
- Balancing class weights and adding more hyperparameters to the Random Forest Classifier has very much improved the precision and recall, as well as stabalized the accuracy of the train set
- Despite the model performing rather alright on the train data after finetuning, the test data set still miserably fails in terms of all performance metrics